In [25]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U tensorflow-text

In [3]:
!pip install -q tf-models-official

In [13]:
!pip install bert-tensorflow


In [4]:
import os, time
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization # to create AdamW optimizer
from kaggle_datasets import KaggleDatasets

from bert import tokenization
print(tf.version.VERSION)

In [7]:
# Set global vairables
sequence_length = 128
data_path = '../input/jigsaw-toxic-severity-rating'
bert_path = '../input/bert-multi'
bert_model_path = '../input/bert-multi/bert_multi_from_tfhub'
output_path = '/kaggle/working'

In [8]:
#Training data from the first competition
# https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/data
wiki_toxic_comment_train = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv')

wiki_toxic_comment_train.head()

In [22]:
# Bert tokenizer
def get_tokenizer(bert_path = bert_model_path):
    '''Tokenizer for a BERT layer'''
    bert_layer = tf.saved_model.load(bert_path)
    bert_layer = hub.KerasLayer(bert_layer, trainable=False)

    vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
    tf.gfile = tf.io.gfile # for bert.tokenization.load_vacab in tokenizer
    tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)
    return tokenizer

tokenizer = get_tokenizer()

In [23]:
#Let's look a example sentenc
example_sentence = wiki_toxic_comment_train.iloc[37].comment_text[:150]
print(example_sentence)

bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

text_preprocessed = bert_preprocess_model(example_sentence)

print(f'Keys : {list(text_preprocessed.keys())}')
print(f'Shape : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids : {text_preprocessed["input_word_ids"][0, :17]}')
print(f'Shape : {text_preprocessed["input_mask"][0, :17]}')
print(f'Shape : {text_preprocessed["input_type_ids"][0. :17]}')